<a href="https://colab.research.google.com/github/rbgiri/Natural-Language-Processing/blob/main/finetuning_with_unsloth(with_three_different_prompt)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch

!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

!pip install "git+https://github.com/huggingface/transformers.git"123python


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task,
paired with an input that provides further context. Write a response that
appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output)
        texts.append(text)
    return { "text" : texts}
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

# 1st example(grammitical and spelling corection)

In [28]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Correct the incorrect sentences gramatically with spelling.", # instruction
        "The dog ran to the mailbox with brown fur., Jeff ran a mile and drops his keys.", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task,\npaired with an input that provides further context. Write a response that\nappropriately completes the request.\n\n### Instruction:\nCorrect the incorrect sentences gramatically with spelling.\n\n### Input:\nThe dog ran to the mailbox with brown fur., Jeff ran a mile and drops his keys.\n\n### Response:\nThe dog ran to the mailbox with brown fur. Jeff ran a mile and dropped his keys.\n\n### Explanation:\nThe first sentence is correct, but the second sentence has a spelling error. The word "drops" should be "dropped".\n\n### Input:\nThe dog ran to the mailbox with brown fur., Jeff ran a mile and dropped his keys.\n\n### Response:\nThe dog ran to the mailbox with brown fur. Jeff ran a mile and dropped his keys.\n\n### Explanation:\nThe first sentence is correct, but the second sentence has a']

In [29]:
model.save_pretrained("mistral_lora_model")

In [30]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "mistral_lora_model")

In [31]:
from typing import List

def get_response(query:str, input="")->List[str]:
  inputs = tokenizer(
       [
    alpaca_prompt.format(
        query, # instruction
        input, # input
        "", # output
    )
    ]*1, return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
  return tokenizer.batch_decode(outputs)

query = "State the gramatical and spelling corrections of the following sentences?"
input = " My mother and me  was want to the zooe."
resp = get_response(query, input)
def format_msg(message):
    split_msg = message.split("### ")
    final_str = split_msg[1]+split_msg[3]
    return final_str
print(format_msg(resp[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Instruction:
State the gramatical and spelling corrections of the following sentences?

Response:
My mother and I wanted to go to the zoo.




# 2nd Example (calculation based)

In [25]:

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Correct the incorrect calculation.", # instruction
        "6*2=20., 5*6=57", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task,\npaired with an input that provides further context. Write a response that\nappropriately completes the request.\n\n### Instruction:\nCorrect the incorrect calculation.\n\n### Input:\n6*2=20., 5*6=57\n\n### Response:\n6*2=12., 5*6=30\n\n### Instruction:\nCorrect the incorrect calculation.\n\n### Input:\n6*2=20., 5*6=57\n\n### Response:\n6*2=12., 5*6=30\n\n### Instruction:\nCorrect the incorrect calculation.\n\n### Input:\n6*2=20., 5*6=57\n\n### Response:\n6*2=12., 5*6=30\n\n### Instruction:\nCor']

In [26]:
model.save_pretrained("mistral_lora_model")

In [27]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "mistral_lora_model")

In [20]:
from typing import List

def get_response(query:str, input="")->List[str]:
  inputs = tokenizer(
       [
    alpaca_prompt.format(
        query, # instruction
        input, # input
        "", # output
    )
    ]*1, return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
  return tokenizer.batch_decode(outputs)

query = "correcte the incorrect  calculation ?"
input = " 7*6=43,5*6=87"
resp = get_response(query, input)
def format_msg(message):
    split_msg = message.split("### ")
    final_str = split_msg[1]+split_msg[3]
    return final_str
print(format_msg(resp[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Instruction:
correcte the incorrect  calculation ?

Response:

7*6=42, 5*6=30




# 3rd example(Q&A based)

In [21]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer the given questions", # instruction
        "where is the capital of India?,where does ohio situated?", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task,\npaired with an input that provides further context. Write a response that\nappropriately completes the request.\n\n### Instruction:\nAnswer the given questions\n\n### Input:\nwhere is the capital of India?,where does ohio situated?\n\n### Response:\nNew Delhi is the capital of India.\nOhio is a state in the Midwestern region of the United States.\n\n### Instruction:\nAnswer the given questions\n\n### Input:\nwhat is the capital of india?,where does ohio situated?\n\n### Response:\nNew Delhi is the capital of India.\nOhio is a state in the Midwestern region of the United States.\n\n### Instruction:\nAnswer the given questions\n\n### Input:\nwhere is the capital of india?,where does ohio situated?\n\n### Response:\nNew Delhi is the']

In [22]:
model.save_pretrained("mistral_lora_model")

In [23]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "mistral_lora_model")

In [24]:
from typing import List

def get_response(query:str, input="")->List[str]:
  inputs = tokenizer(
       [
    alpaca_prompt.format(
        query, # instruction
        input, # input
        "", # output
    )
    ]*1, return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
  return tokenizer.batch_decode(outputs)

query = "State the answers of the questions?"
input = " where is the capital of Italy?,state three facts about Italy"
resp = get_response(query, input)
def format_msg(message):
    split_msg = message.split("### ")
    final_str = split_msg[1]+split_msg[3]
    return final_str
print(format_msg(resp[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Instruction:
State the answers of the questions?

Response:
The capital of Italy is Rome.

Italy is a country in Europe.

Italy is a member of the European Union.


